#### Please read the [docs](https://ipywidgets.readthedocs.io/en/stable/user_install.html) for `ipywidgets` on how to install them.

In [1]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from matplotlib import pyplot as plt
import numpy as np
from cmocean import cm

After importing the required modules, we need to construct some data to plot

In [2]:
# construct 2D data
some_sin = np.array(np.sin(np.linspace(0, np.pi, 20)))
some_cos = np.array(np.cos(np.linspace(0, np.pi, 20)))
some_data = some_sin[:, None] * some_cos[None, :]

# add depth dimension of length `z`
z = 8
some_data_with_depth = some_data[None, :, :] * np.exp(np.linspace(1, 2, z))[:, None, None]

# add time dimension of length `t`
t = 10
data = some_data_with_depth[None, :, :, :] * np.cos(np.linspace(0, np.pi / 2, t))[:, None, None, None]

Say, additionally to explore the data we also want to compare the colormaps of `matplotlib` and `cmocean`. So first we build two widgets, were the second (to choose the colormap) depends on the first (to choose between `matplotlib` and `cmocean`)  

This is a bit tricky and we have to put the colormaps into dictionaries for this to work. (If you want to compare colormaps of different packages more often, you can have a look at this [repository](https://github.com/jk-rieck/colormaps_from_palettable))

In [5]:
# First, we put the colormaps of the two different packages into a dictionary
cmaps = {}
cmaps['matplotlib'] = {}
cmaps['cmocean'] = {}

for m in plt.colormaps():
    if m[0:3] != 'cmo':
        cmaps['matplotlib'][m] = getattr(plt.cm, m)
        
for n in plt.colormaps():
    if n[0:3] == 'cmo':
        n_strp = n[4::]
        cmaps['cmocean'][n_strp] = getattr(cm, n_strp)

widget1 = widgets.Dropdown(options = {i: j for (i, j) in cmaps.items()})
widget2 = widgets.Dropdown()

def update(*args):
    widget2.options = {k: l for (k, l) in widget1.value.items()}

widget1.observe(update)

Now construct the interactive plot. We also want to be able to change the time `t`, the depth `z`, as well as the minimum and maximum visible values `vmin` and `vmax`.

In [7]:
@interact
def contour_plot(pack=widget1,
                 cmap=widget2,
                 time=widgets.IntSlider(min=0, max=t-1, value=0),
                 depth=widgets.IntSlider(min=0, max=z-1, value=0),
                 vmin=widgets.FloatSlider(min=-8., max=8., step=0.2, value=-6.),
                 vmax=widgets.FloatSlider(min=-8., max=8., step=0.2, value=6.)):
    plt.pcolormesh(data[time, depth, :, :], cmap=cmap, vmin=vmin, vmax=vmax)
    plt.colorbar()

interactive(children=(Dropdown(description='pack', index=1, options={'matplotlib': {'Accent': <matplotlib.colo…

In [ ]:
plt.cm.items()